In [1]:
import findspark 
findspark.init()

In [2]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession\
    .builder\
    .master('local[2]')\
    .appName('quake_etl')\
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:2.4.1')\
    .getOrCreate()


In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
# load the dataset 
df_load = spark.read.csv(r"C:\Users\ghita\Downloads\database.csv", header=True)
# Preview df_load
df_load.take(1)

In [4]:
# drop fields we don t need from df_load 
lst_dropped_columns = ['Depth Error', 'Time', 'Depth Seismic Stations','Magnitude Error','Magnitude Seismic Stations','Azimuthal Gap', 'Horizontal Distance','Horizontal Error',
    'Root Mean Square','Source','Location Source','Magnitude Source','Status']
df_load = df_load.drop(*lst_dropped_columns)
# Preview df_load
df_load.show(5)

+----------+--------+---------+----------+-----+---------+--------------+------------+
|      Date|Latitude|Longitude|      Type|Depth|Magnitude|Magnitude Type|          ID|
+----------+--------+---------+----------+-----+---------+--------------+------------+
|01/02/1965|  19.246|  145.616|Earthquake|131.6|        6|            MW|ISCGEM860706|
|01/04/1965|   1.863|  127.352|Earthquake|   80|      5.8|            MW|ISCGEM860737|
|01/05/1965| -20.579| -173.972|Earthquake|   20|      6.2|            MW|ISCGEM860762|
|01/08/1965| -59.076|  -23.557|Earthquake|   15|      5.8|            MW|ISCGEM860856|
|01/09/1965|  11.938|  126.427|Earthquake|   15|      5.8|            MW|ISCGEM860890|
+----------+--------+---------+----------+-----+---------+--------------+------------+
only showing top 5 rows



In [5]:
# Create a year field and add it to the dataframe 

df_load = df_load.withColumn('Year', year(to_timestamp('Date', 'dd/MM/yyyy')))
# Preview df_load

df_load.show(5)


+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|      Date|Latitude|Longitude|      Type|Depth|Magnitude|Magnitude Type|          ID|Year|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
|01/02/1965|  19.246|  145.616|Earthquake|131.6|        6|            MW|ISCGEM860706|1965|
|01/04/1965|   1.863|  127.352|Earthquake|   80|      5.8|            MW|ISCGEM860737|1965|
|01/05/1965| -20.579| -173.972|Earthquake|   20|      6.2|            MW|ISCGEM860762|1965|
|01/08/1965| -59.076|  -23.557|Earthquake|   15|      5.8|            MW|ISCGEM860856|1965|
|01/09/1965|  11.938|  126.427|Earthquake|   15|      5.8|            MW|ISCGEM860890|1965|
+----------+--------+---------+----------+-----+---------+--------------+------------+----+
only showing top 5 rows



In [6]:
# Build the quakes frequency dataframe using the year field and counts for each year 

df_quake_freq = df_load.groupBy('Year').count().withColumnRenamed('count','Counts')

# Preview df_quake_freq

df_quake_freq.show(5)

+----+------+
|Year|Counts|
+----+------+
|1990|   196|
|1975|   150|
|1977|   148|
|2003|   187|
|2007|   211|
+----+------+
only showing top 5 rows



In [7]:
# Preview df_load_schema 
df_load.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Depth: string (nullable = true)
 |-- Magnitude: string (nullable = true)
 |-- Magnitude Type: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Year: integer (nullable = true)



In [8]:
# Cast some fields from string into numeric types 

df_load = df_load.withColumn('Latitude', df_load['Latitude'].cast(DoubleType()))\
    .withColumn('Longitude', df_load['Longitude'].cast(DoubleType()))\
    .withColumn('Depth', df_load['Depth'].cast(DoubleType()))\
    .withColumn('Magnitude', df_load['Magnitude'].cast(DoubleType()))

# Preview df_load

df_load.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Type: string (nullable = true)
 |-- Depth: double (nullable = true)
 |-- Magnitude: double (nullable = true)
 |-- Magnitude Type: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Year: integer (nullable = true)



In [9]:
# Create avg magnitude and max magnitude and add to df_quake_freq

df_max = df_load.groupBy('Year').max('Magnitude').withColumnRenamed('max(Magnitude)','Max_Magnitude')
df_avg = df_load.groupBy('Year').avg('Magnitude').withColumnRenamed('avg(Magnitude)', 'Avg_Magnitude')
df_avg.show(5)

+----+-----------------+
|Year|    Avg_Magnitude|
+----+-----------------+
|1990|5.858163265306125|
|1975| 5.84866666666667|
|1977|5.757432432432437|
|2003|5.850802139037435|
|2007| 5.89099526066351|
+----+-----------------+
only showing top 5 rows



In [10]:
# join df_max and df_avg to df_quake_freq

df_quake_freq = df_quake_freq.join(df_avg,['Year']).join(df_max,['Year'])

#Preview df_quake_freq

df_quake_freq.show(5)

+----+------+-----------------+-------------+
|Year|Counts|    Avg_Magnitude|Max_Magnitude|
+----+------+-----------------+-------------+
|1990|   196|5.858163265306125|          7.6|
|1975|   150| 5.84866666666667|          7.8|
|1977|   148|5.757432432432437|          7.6|
|2003|   187|5.850802139037435|          7.6|
|2007|   211| 5.89099526066351|          8.4|
+----+------+-----------------+-------------+
only showing top 5 rows



In [11]:
# Remove null

df_load.dropna()
df_quake_freq.dropna()

DataFrame[Year: int, Counts: bigint, Avg_Magnitude: double, Max_Magnitude: double]

In [14]:
#Build the tables/collections in mongodb
#write df_load to mongodb

df_load.write.format('mongo')\
     .mode('overwrite')\
     .option('spark.mongodb.output.uri', 'mongodb://localhost:27017/Quake.quakes').save()

In [15]:
# write df_quake_freq to mongodb

df_quake_freq.write.format('mongo')\
     .mode('overwrite')\
     .option('spark.mongodb.output.uri', 'mongodb://localhost:27017/Quake.quake_freq').save()

# Section: Machine Learning with Spark



In [4]:
# Load the test data into a dataframe 

df_test = spark.read.csv(r"C:\Users\ghita\Downloads\query.csv", header=True)

# Preview df_test
df_test.take(1)

[Row(time='2017-01-02T00:13:06.300Z', latitude='-36.0365', longitude='51.9288', depth='10', mag='5.7', magType='mwb', nst=None, gap='26', dmin='14.685', rms='1.37', net='us', id='us10007p5d', updated='2017-03-27T23:53:17.040Z', place='Southwest Indian Ridge', type='earthquake', horizontalError='10.3', depthError='1.7', magError='0.068', magNst='21', status='reviewed', locationSource='us', magSource='us')]

In [5]:
# Load the training data from mongo into a dataframe

df_train = spark.read.format('mongo')\
   .option('spark.mongodb.input.uri','mongodb://localhost:27017/Quake.quakes').load()

# Preview df_train

df_train.show(5)

+----------+-----+------------+--------+---------+---------+--------------+----------+----+--------------------+
|      Date|Depth|          ID|Latitude|Longitude|Magnitude|Magnitude Type|      Type|Year|                 _id|
+----------+-----+------------+--------+---------+---------+--------------+----------+----+--------------------+
|01/02/1965|131.6|ISCGEM860706|  19.246|  145.616|      6.0|            MW|Earthquake|1965|{65b7b9f56ed8fd5c...|
|01/04/1965| 80.0|ISCGEM860737|   1.863|  127.352|      5.8|            MW|Earthquake|1965|{65b7b9f56ed8fd5c...|
|01/05/1965| 20.0|ISCGEM860762| -20.579| -173.972|      6.2|            MW|Earthquake|1965|{65b7b9f56ed8fd5c...|
|01/08/1965| 15.0|ISCGEM860856| -59.076|  -23.557|      5.8|            MW|Earthquake|1965|{65b7b9f56ed8fd5c...|
|01/09/1965| 15.0|ISCGEM860890|  11.938|  126.427|      5.8|            MW|Earthquake|1965|{65b7b9f56ed8fd5c...|
+----------+-----+------------+--------+---------+---------+--------------+----------+----+-----

In [6]:
# Select fields we will use and discard fields we don't need

df_test_clean = df_test['time','latitude','longitude','mag','depth']
# Preview df_test_clean 
df_test_clean.show(5)

+--------------------+--------+---------+---+------+
|                time|latitude|longitude|mag| depth|
+--------------------+--------+---------+---+------+
|2017-01-02T00:13:...|-36.0365|  51.9288|5.7|    10|
|2017-01-02T13:13:...|  -4.895| -76.3675|5.9|   106|
|2017-01-02T13:14:...|-23.2513| 179.2383|6.3|551.62|
|2017-01-03T09:09:...| 24.0151|  92.0177|5.7|    32|
|2017-01-03T21:19:...|-43.3527| -74.5017|5.5| 10.26|
+--------------------+--------+---------+---+------+
only showing top 5 rows



In [9]:
# Rename fields
df_test_clean = df_test_clean.withColumnRenamed('time', 'Date')\
     .withColumnRenamed('latitude', 'Latitude')\
     .withColumnRenamed('longitude', 'Longitude')\
     .withColumnRenamed('mag', 'Magnitude')\
     .withColumnRenamed('depth', 'Depth')\

df_test_clean.show(5)

+--------------------+--------+---------+---------+------+
|                Date|Latitude|Longitude|Magnitude| Depth|
+--------------------+--------+---------+---------+------+
|2017-01-02T00:13:...|-36.0365|  51.9288|      5.7|    10|
|2017-01-02T13:13:...|  -4.895| -76.3675|      5.9|   106|
|2017-01-02T13:14:...|-23.2513| 179.2383|      6.3|551.62|
|2017-01-03T09:09:...| 24.0151|  92.0177|      5.7|    32|
|2017-01-03T21:19:...|-43.3527| -74.5017|      5.5| 10.26|
+--------------------+--------+---------+---------+------+
only showing top 5 rows



In [11]:
df_test_clean.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Magnitude: string (nullable = true)
 |-- Depth: string (nullable = true)



In [14]:
# Cast some fields into numerical fields 
df_test_clean = df_test_clean.withColumn('Latitude', df_test_clean['Latitude'].cast(DoubleType()))\
      .withColumn('Longitude', df_test_clean['Longitude'].cast(DoubleType()))\
      .withColumn('Depth', df_test_clean['Depth'].cast(DoubleType()))\
      .withColumn('Magnitude', df_test_clean['Magnitude'].cast(DoubleType()))\
      

In [15]:
df_test_clean.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Magnitude: double (nullable = true)
 |-- Depth: double (nullable = true)



In [16]:
# Create testing and training dataframes 
df_testing = df_test_clean['Latitude','Longitude','Magnitude','Depth']
df_training = df_train['Latitude','Longitude','Magnitude','Depth']

In [18]:
df_testing.printSchema()

root
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Magnitude: double (nullable = true)
 |-- Depth: double (nullable = true)



In [19]:
# Drop records with null values from our dataframe 
df_testing = df_testing.dropna()
df_training = df_training.dropna()

In [22]:
# Select features to parse into our model then create the feature vector 
assembler = VectorAssembler(inputCols = ['Latitude','Longitude','Depth'], outputCol='features')

# Create the Model
model_reg = RandomForestRegressor(featuresCol='features', labelCol='Magnitude')

# Chain the assembler with the model in a pipeline 
pipeline = Pipeline(stages=[assembler, model_reg])

# Train the model

model = pipeline.fit(df_training)

# Make the prediction on testing data

pred_results = model.transform(df_testing)


In [23]:
# Preview pred_results dataframe 
pred_results.show(5)

+--------+---------+---------+------+--------------------+-----------------+
|Latitude|Longitude|Magnitude| Depth|            features|       prediction|
+--------+---------+---------+------+--------------------+-----------------+
|-36.0365|  51.9288|      5.7|  10.0|[-36.0365,51.9288...|5.857693314646372|
|  -4.895| -76.3675|      5.9| 106.0|[-4.895,-76.3675,...|   5.891439643237|
|-23.2513| 179.2383|      6.3|551.62|[-23.2513,179.238...|5.869618330110986|
| 24.0151|  92.0177|      5.7|  32.0|[24.0151,92.0177,...|5.921863245148279|
|-43.3527| -74.5017|      5.5| 10.26|[-43.3527,-74.501...|5.950561153197301|
+--------+---------+---------+------+--------------------+-----------------+
only showing top 5 rows



In [24]:
# Evaluate the model 
# rmse should be less than 0.5 for the model to be useful 

evaluator = RegressionEvaluator(labelCol ='Magnitude', predictionCol='prediction',metricName='rmse')
rmse = evaluator.evaluate(pred_results)
print(rmse)

0.4041968304469984


# Create the prediction dataset 

In [25]:
# Create the prediction dataset 
df_pred_results = pred_results['Latitude','Longitude','prediction']

# rename the prediction field 

df_pred_results = df_pred_results.withColumnRenamed('prediction','Pred_Magnitude')

#add more columns to our prediction dataset 
df_pred_results = df_pred_results.withColumn('Year',lit(2017))\
       .withColumn('RMSE',lit(rmse))

# Preview the prediction dataset 
df_pred_results.show(5)


+--------+---------+-----------------+----+------------------+
|Latitude|Longitude|   Pred_Magnitude|Year|              RMSE|
+--------+---------+-----------------+----+------------------+
|-36.0365|  51.9288|5.857693314646372|2017|0.4041968304469984|
|  -4.895| -76.3675|   5.891439643237|2017|0.4041968304469984|
|-23.2513| 179.2383|5.869618330110986|2017|0.4041968304469984|
| 24.0151|  92.0177|5.921863245148279|2017|0.4041968304469984|
|-43.3527| -74.5017|5.950561153197301|2017|0.4041968304469984|
+--------+---------+-----------------+----+------------------+
only showing top 5 rows



In [26]:
# Load the prediction dataset into mongodb

# Write df_pred_results
df_pred_results.write.format('mongo')\
     .mode('overwrite')\
     .option('spark.mongodb.output.uri', 'mongodb://localhost:27017/Quake.pred_results').save()
                

# Section : Data visualization

In [4]:
import pandas as pd
from bokeh.io import output_notebook, output_file
from bokeh.plotting import figure, show, ColumnDataSource
import math
from math import pi 
from bokeh.palettes import Category20c
from bokeh.transform import cumsum
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.themes import built_in_themes
from bokeh.io import curdoc
from pymongo import MongoClient


In [6]:
pip install pymongo

  Obtaining dependency information for pymongo from https://files.pythonhosted.org/packages/c8/1a/1ba5578cc3acc8ef0dfd6eb9385f9c4b0fe3039665af93bcf062c34fcdb6/pymongo-4.6.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for dnspython<3.0.0,>=1.16.0 from https://files.pythonhosted.org/packages/b6/83/4a684a63d395007670bc95c1947c07045fe66141574e2f7e9e347df8499a/dnspython-2.5.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/472.7 kB ? eta -:--:--
   --- ----------------------------------- 41.0/472.7 kB 653.6 kB/s eta 0:00:01
   ----------------- ---------------------- 204.8/472.7 kB 2.1 MB/s eta 0:00:01
   ----------------------- ---------------- 276.5/472.7 kB 2.4 MB/s eta 0:00:01
   -------------------------------- ------- 389.1/472.7 kB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 472.7/472.7 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/305.4 kB ? eta -:--:--
   --------------------- -------

In [5]:
# Create custom read function to read data from mongodb into a dataframe

def read_mongo(host='127.0.0.1', port=27017, username=None, password=None,db='Quake', collection='pred_results'):
    
    mongo_uri = 'mongodb://{}:{}/{}.{}'.format(host,port,db,collection)
    # Connect to mongodb
    conn = MongoClient(mongo_uri)
    db = conn[db]
    
    # Select all records from the collection 
    cursor = db[collection].find()
    
    # Create the dataframe
    df = pd.DataFrame(list(cursor))
    
    # Delete the _id field 
    del df['_id']
    
    return df

In [6]:
# Load the dataset from mongodb 
df_quakes = read_mongo(collection='quakes')
df_quake_freq = read_mongo(collection='quake_freq')
df_quake_pred = read_mongo(collection='pred_results')

In [11]:
df_quakes.head()

,Date,Latitude,Longitude,Type,Depth,Magnitude,Magnitude Type,ID,Year
0,01/02/1965,19.246,145.616,Earthquake,131.6,6.0,MW,ISCGEM860706,1965.0
1,01/04/1965,1.863,127.352,Earthquake,80.0,5.8,MW,ISCGEM860737,1965.0
2,01/05/1965,-20.579,-173.972,Earthquake,20.0,6.2,MW,ISCGEM860762,1965.0
3,01/08/1965,-59.076,-23.557,Earthquake,15.0,5.8,MW,ISCGEM860856,1965.0
4,01/09/1965,11.938,126.427,Earthquake,15.0,5.8,MW,ISCGEM860890,1965.0


In [7]:
df_quakes_2016 = df_quakes[df_quakes['Year'] == 2016]
df_quakes_2016.head()

,Date,Latitude,Longitude,Type,Depth,Magnitude,Magnitude Type,ID,Year
22943,01/01/2016,-50.5575,139.4489,Earthquake,10.00,6.3,MWW,US10004ANT,2016.0
22944,01/01/2016,-28.6278,-177.2810,Earthquake,34.00,5.8,MWW,US10004AQY,2016.0
22945,01/02/2016,44.8069,129.9406,Earthquake,585.47,5.8,MWW,US10004ATB,2016.0
22946,01/03/2016,24.8036,93.6505,Earthquake,55.00,6.7,MWW,US10004B2N,2016.0
22947,01/05/2016,30.6132,132.7337,Earthquake,4.71,5.8,MWW,US10004BEN,2016.0


In [8]:
# Show plots embedded in jupyter notebook 
output_notebook()

Loading BokehJS ...

In [9]:
# Create custom style function to style our plots 

def style(p):
    #Title 
    p.title.align = 'center'
    p.title.text_font_size = '20pt'
    p.title.text_font = 'serif'
    
    # Add titles 
    p.xaxis.axis_label_text_font_size = '14pt'
    p.xaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = '14pt'
    p.yaxis.axis_label_text_font_style = 'bold'
    
    # Tick labels 
    p.xaxis.major_label_text_font_size = '12pt'
    p.yaxis.major_label_text_font_size = '12pt'
    
    # Plot the legend in the top left corner 
    p.legend.location = 'top_left'
    
    return p

    

In [72]:
# Create the Geo Map plot 
def plotMap():
    lat = df_quakes_2016['Latitude'].values.tolist()
    lon = df_quakes_2016['Longitude'].values.tolist()
    
    
    pred_lat = df_quake_pred['Latitude'].values.tolist()
    pred_lon = df_quake_pred['Longitude'].values.tolist()
    
    lst_lat = []
    lst_lon = []
    lst_pred_lat = []
    lst_pred_lon = []
    
    i = 0
    j = 0
    
    # Convert Lat and Long values into merc_projection format 
# Convert Lat and Long values into merc_projection format
    for i in range(len(lon)):
        r_major = 6378137.000
        x = r_major * math.radians(lon[i])
        scale = x/lon[i]
        y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 +
            lat[i] * (math.pi/180.0)/2.0)) * scale
        
        lst_lon.append(x)
        lst_lat.append(y)
        i += 1
        
    # Convert predicted lat and long values into merc_projection format
    for j in range(len(pred_lon)):
        r_major = 6378137.000
        x = r_major * math.radians(pred_lon[j])
        scale = x/pred_lon[j]
        y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 +
            pred_lat[j] * (math.pi/180.0)/2.0)) * scale
        
        lst_pred_lon.append(x)
        lst_pred_lat.append(y)
        j += 1
        
    
    df_quakes_2016['coords_x'] = lst_lat
    df_quakes_2016['coords_y'] = lst_lon
    df_quake_pred['coords_x'] = lst_pred_lat
    df_quake_pred['coords_y'] = lst_pred_lon
    
    # Scale the circles 
    df_quakes_2016['Mag_Size'] = df_quakes_2016['Magnitude'] * 4
    df_quake_pred['Mag_Size'] = df_quake_pred['Pred_Magnitude'] * 4
    
    # Create datasources for our ColumnDataSource object 
    lats = df_quakes_2016['coords_x'].tolist()
    longs = df_quakes_2016['coords_y'].tolist()
    mags = df_quakes_2016['Magnitude'].tolist()
    years = df_quakes_2016['Year'].tolist()
    mag_size = df_quakes_2016['Mag_Size'].tolist()
    
    pred_lats = df_quake_pred['coords_x'].tolist()
    pred_longs = df_quake_pred['coords_y'].tolist()
    pred_mags = df_quake_pred['Pred_Magnitude'].tolist()
    pred_year = df_quake_pred['Year'].tolist()
    pred_mag_size = df_quake_pred['Mag_Size'].tolist()
    
    
    # Create column datasource 
    
    cds = ColumnDataSource(
    data = dict(
        lat = lats,
        lon = longs,
        mag = mags,
        year = years,
        mag_s = mag_size
    )
    )
    
    pred_cds = ColumnDataSource(
    data = dict(
        pred_lat = pred_lats,
        pred_lon = pred_longs,
        pred_mag = pred_mags,
        year = pred_year,
        pred_mag_s = pred_mag_size
    )
    )
    
    # Tooltips 
    TOOLTIPS = [
        ("Year", "@year"),
        ("Magnitude", "@mag"),
        ("Predicted Magnitude", " @pred_mag")
    ]
    # Create figure 
    
    # Create figure
    p = figure(title='Earthquake Map',
              width=2300, height=850,
              x_range=(-2000000, 6000000),
              y_range=(-1000000, 7000000),
              tooltips=TOOLTIPS)
    p.circle(x='lon',y='lat', size='mag_s', fill_color='#cc0000', fill_alpha = 0.7,
            source = cds, legend_label ='Quakes 2016')
    # Add circles for our predicted earthquakes 
    
    p.circle(x = 'pred_lon', y ='pred_lat', size='pred_mag_s', fill_color='#ccff33', fill_alpha = 7.0,
            source = pred_cds, legend_label ='Quakes 2017')
    
    p.add_tile(CARTODBPOSITRON)
    
    
    # Style the map plot
    # Title
    p.title.align='center'
    p.title.text_font_size='20pt'
    p.title.text_font='serif'
    
    # Legend
    p.legend.location='bottom_right'
    p.legend.background_fill_color='black'
    p.legend.background_fill_alpha=0.8
    p.legend.click_policy='hide'
    p.legend.label_text_color='white'
    p.xaxis.visible=False
    p.yaxis.visible=False
    p.axis.axis_label=None
    p.axis.visible=False
    p.grid.grid_line_color=None
    return p 
    

In [44]:
# Create the bar chart 
def plotBar() :
    # Load the datasource 
    cds = ColumnDataSource(data = dict(
    yrs = df_quake_freq['Year'].values.tolist(),
    numQuakes = df_quake_freq['Counts'].values.tolist()
    ))
    # Tooltip 
    TOOLTIPS = [
        ('Year','@yrs'),
        ('Number of earthquakes', '@numQuakes')
    ]
    # Create a figure 
    barChart = figure(title = 'Frequency of Earthquakes by Year',
                     height = 400,
                     width = 1150,
                     x_axis_label = 'Year',
                     y_axis_label = 'Number of Occurances',
                     x_minor_ticks = 2,
                     y_range = (0,df_quake_freq['Counts'].max()+100),
                     toolbar_location = None,
                      tooltips = TOOLTIPS
                     )
    # Create a vertical bar 
    barChart.vbar(x='yrs', bottom = 0, top = 'numQuakes',
                 color = '#cc0000', width = 0.75,
                 legend_label = 'Year', source = cds)
    
    #Style the bar chart 
    
    barChart = style(barChart)
    return barChart

In [43]:
# Create a magnitude plot

def plotMagnitude() :
    #Load the datasource 
    cds = ColumnDataSource(data = dict(
    yrs = sorted(df_quake_freq['Year'].values.tolist()),
    avg_mag = df_quake_freq['Avg_Magnitude'].values.tolist(),
    max_mag = df_quake_freq['Max_Magnitude'].values.tolist()
    ))
    TOOLTIPS = [
        ('Year','@yrs'),
        ('Average Magnitude', '@avg_mag'),
        ('Maximum Magnitude', '@max_mag')
    ]
    
    # Create a figure 
    mp = figure(title = 'Maximum and Average Magnitude by Year',
    width = 1150, height = 400,
    x_axis_label = 'Years',
    y_axis_label = 'Magnitude',
    x_minor_ticks = 2,
    y_range = (5, df_quake_freq['Max_Magnitude'].max() + 1 ),
    toolbar_location = None,
    tooltips = TOOLTIPS
    )
    # Create the line for the Max Magnitude 
    # Max Magnitude
    mp.line(x='yrs', y='max_mag', color='#cc0000', line_width=2, legend_label='Max Magnitude', source=cds)
    mp.circle(x='yrs', y='max_mag', color='#cc0000', size=8, fill_color='#cc0000', source=cds)
    
    # Average Magnitude 
    mp.line(x='yrs', y='avg_mag', color='yellow', line_width=2, legend_label='Avg Magnitude', source=cds)
    mp.circle(x='yrs', y='avg_mag', color='yellow', size=8, fill_color='yellow', source=cds)
    mp = style(mp)
    return mp 

In [19]:
df_quake_freq.head()

,Year,Counts,Avg_Magnitude,Max_Magnitude
0,1990,196,5.858163,7.6
1,1975,150,5.848667,7.8
2,1977,148,5.757432,7.6
3,2003,187,5.850802,7.6
4,2007,211,5.890995,8.4


figure(id='p1742', ...)

In [53]:
# Display the visuals in the browser 

output_file('dashboard.html')

# Change to a dark theme
curdoc().theme = 'dark_minimal'

In [73]:
# Build the grid plot 
from bokeh.layouts import gridplot

# Create the grid 

grid = gridplot([[plotMap()],[plotMagnitude()],[plotBar()]], sizing_mode='scale_width')

# Show the grid 

show(grid)


C:\Users\ghita\AppData\Local\Temp\ipykernel_15852\1752945502.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quakes_2016['coords_x'] = lst_lat
C:\Users\ghita\AppData\Local\Temp\ipykernel_15852\1752945502.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_quakes_2016['coords_y'] = lst_lon
C:\Users\ghita\AppData\Local\Temp\ipykernel_15852\1752945502.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in